# Google Maps Review Summarizer - Day 2 Solution

This notebook demonstrates web scraping Google Maps reviews and analyzing them with Llama 3.2 via Ollama.

## Features:
- Web scraping with Playwright
- Async data extraction
- LLM-based sentiment analysis
- Local LLM (no API costs!)

**Author:** İrem Bezci (r00trose)  
**Course:** LLM Engineering - Week 1, Day 2

## Setup & Installation

First, make sure you have the required packages installed:

In [ ]:
# Install required packages (run once)
!pip install playwright ollama
!playwright install chromium

## Import Libraries

In [ ]:
import asyncio
import ollama
from playwright.async_api import async_playwright
from IPython.display import Markdown, display

## Helper Functions

### 1. Scroll Reviews Panel
This function scrolls through the reviews to load more content.

In [ ]:
async def scroll_reviews_panel(page, max_scrolls=10, max_reviews=20):
    """Scrolls through reviews panel to load all reviews."""
    scrollable_div = page.locator('div[role="main"] div[jslog$="mutable:true;"]').first
    previous_review_count = 0
    scroll_attempts = 0
    no_change_count = 0
    
    print("📜 Scrolling to load reviews...")
    
    while scroll_attempts < max_scrolls:
        review_elements = page.locator("div[data-review-id][jsaction]")
        current_review_count = await review_elements.count()
        
        if current_review_count >= max_reviews:
            break
        
        print(f"   Scroll {scroll_attempts + 1}/{max_scrolls}: Found {current_review_count} reviews")
        
        await scrollable_div.evaluate("""
            (element) => {
                element.scrollTo(0, element.scrollHeight + 100);
            }
        """)
        
        await asyncio.sleep(2)
        
        if current_review_count == previous_review_count:
            no_change_count += 1
            if no_change_count >= 3:
                print(f"   No new reviews loaded. Finished.")
                break
        else:
            no_change_count = 0
        
        previous_review_count = current_review_count
        scroll_attempts += 1
    
    final_count = await review_elements.count()
    print(f"✅ Loaded {final_count} reviews!\n")
    return final_count

### 2. Scrape Google Reviews
Main scraping function using Playwright.

In [ ]:
async def scrape_google_reviews(url, max_reviews=20):
    """Scrape reviews from Google Maps URL."""
    reviews = []
    
    async with async_playwright() as p:
        print("🌐 Opening Google Maps...")
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context()
        page = await context.new_page()
        
        await page.goto(url)
        
        print("⏳ Waiting for reviews to load...")
        review_html_elements = page.locator("div[data-review-id][jsaction]")
        
        try:
            await review_html_elements.first.wait_for(state="visible", timeout=10000)
        except:
            print("❌ Could not find reviews on this page.")
            await browser.close()
            return []
        
        # Scroll to load more reviews
        total_reviews = await scroll_reviews_panel(page, max_scrolls=10, max_reviews=max_reviews)
        
        print("📥 Extracting review data...")
        review_html_elements = page.locator("div[data-review-id][jsaction]")
        all_reviews = await review_html_elements.all()
        
        for idx, review_html_element in enumerate(all_reviews, 1):
            try:
                # Extract rating
                stars_element = review_html_element.locator("[aria-label*=\"star\"]")
                stars_label = await stars_element.get_attribute("aria-label")
                
                stars = None
                for i in range(1, 6):
                    if stars_label and str(i) in stars_label:
                        stars = i
                        break
                
                # Expand "More" button if present
                more_element = review_html_element.locator("button[aria-label=\"See more\"]").first
                if await more_element.is_visible():
                    await more_element.click()
                    await asyncio.sleep(0.3)
                
                # Extract review text
                text_element = review_html_element.locator("div[tabindex=\"-1\"][id][lang]")
                text = await text_element.text_content()
                
                reviews.append({
                    'rating': stars,
                    'text': text.strip()
                })
                
                if idx % 5 == 0:
                    print(f"   Extracted {idx}/{total_reviews} reviews...")
            
            except Exception as e:
                continue
        
        await browser.close()
        
        print(f"✅ Successfully extracted {len(reviews)} reviews!\n")
        return reviews

### 3. Analyze Reviews with LLM
Uses Llama 3.2 via Ollama to analyze sentiment and extract insights.

In [ ]:
def analyze_reviews(reviews, model="llama3.2"):
    """Analyze and summarize reviews using Llama 3.2."""
    
    if not reviews:
        return "No reviews to analyze."
    
    # Prepare reviews text
    reviews_text = "\n\n".join([
        f"Rating: {r['rating']}/5\n{r['text']}" 
        for r in reviews
    ])
    
    # Limit text length
    reviews_text = reviews_text[:8000]
    
    prompt = f"""Analyze the following Google Maps reviews and provide:

1. **Overall Sentiment**: Positive, Negative, or Mixed
2. **Key Themes**: Main topics mentioned
3. **Pros**: What customers love (3-5 bullet points)
4. **Cons**: What customers dislike (3-5 bullet points)
5. **Summary**: A 2-3 sentence overall summary

REVIEWS:
{reviews_text}
"""
    
    print("🧠 Analyzing reviews with Llama 3.2...")
    print("   (This may take a moment...)\n")
    
    try:
        response = ollama.chat(
            model=model,
            messages=[
                {
                    "role": "system",
                    "content": "You are an expert at analyzing customer reviews and extracting insights."
                },
                {
                    "role": "user",
                    "content": prompt
                }
            ]
        )
        return response["message"]["content"]
    
    except Exception as e:
        return f"❌ Analysis error: {e}"

## Example Usage

### Test with Sultanahmet Mosque (Blue Mosque) in Istanbul

In [ ]:
# Example URL - Sultanahmet Mosque
url = "https://www.google.com/maps/place/Sultanahmet+Camii/@41.0054096,28.9768138,17z"

# Number of reviews to scrape
max_reviews = 15

# Scrape reviews
reviews = await scrape_google_reviews(url, max_reviews=max_reviews)

### Display Sample Reviews

In [ ]:
print(f"\n📝 Sample Reviews ({min(3, len(reviews))} of {len(reviews)}):")
print("=" * 60)

for i, review in enumerate(reviews[:3], 1):
    rating_stars = "⭐" * (review['rating'] or 0)
    print(f"\n{i}. {rating_stars}")
    print(f"   {review['text'][:200]}...")

### Analyze Reviews with Llama 3.2

In [ ]:
# Analyze the reviews
analysis = analyze_reviews(reviews)

# Display as formatted markdown
display(Markdown(f"## 📊 Review Analysis\n\n{analysis}"))

## Try Your Own Location!

Replace the URL below with any Google Maps place:

In [ ]:
# Your custom URL
custom_url = "https://www.google.com/maps/place/YOUR_PLACE_HERE"

# Scrape and analyze
custom_reviews = await scrape_google_reviews(custom_url, max_reviews=20)
custom_analysis = analyze_reviews(custom_reviews)

display(Markdown(f"## 📊 Analysis\n\n{custom_analysis}"))

## What I Learned

- **Web Scraping**: Using Playwright for dynamic content
- **Async Programming**: Handling asynchronous operations in Python
- **DOM Navigation**: Finding and extracting specific HTML elements
- **LLM Integration**: Using Llama 3.2 for sentiment analysis
- **Prompt Engineering**: Crafting effective analysis prompts
- **Data Processing**: Structuring and cleaning scraped data